In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries and modules
import os
from PIL import Image
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter

# Function to convert into grayscale image
def gray_scaling(image):
  image_gray = image.convert('L')
  return image_gray

# Function to crop non brain portion from image
def cropping(image_gray):

  # Convert the image to a NumPy array
  image_gray = np.array(image_gray)

  # Threshold the image to separate brain and non-brain regions
  _, thresholded = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  # Find the contours of the brain region
  contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Find the largest contour
  max_contour = max(contours, key=cv2.contourArea)

  # Determine the bounding box of the brain region
  x, y, w, h = cv2.boundingRect(max_contour)

  # Crop the image based on the bounding box
  image_cropped = image_gray[y:y+h, x:x+w]

  # Convert NumPy array to image object
  image_cropped = Image.fromarray(image_cropped)

  return image_cropped

# Function to rescale image
def rescaling(image_cropped):
  image_rescaled = image_cropped.resize((224, 224))
  return image_rescaled

# Function to normalize intensity
def normalize_intensity(image_rescaled):
    image_array = np.array(image_rescaled)
    image_array = image_array.astype(float)

    # Perform intensity normalization
    min_val = np.min(image_array)
    max_val = np.max(image_array)
    image_normalized = (image_array - min_val) / (max_val - min_val)

    image_normalized = Image.fromarray((image_normalized * 255).astype(np.uint8))
    return image_normalized

# Function to apply gaussian filter
def filter_gaussian(image_normalized):
    image_array = np.array(image_normalized)

    # Apply Gaussian filter
    sigma = 1.3
    filtered_array = gaussian_filter(image_array, sigma)

    image_filtered = Image.fromarray(filtered_array.astype(np.uint8))
    return image_filtered

# Function to stretch contrast
def contrast_stretch(image_filtered):
    image_array = np.array(image_filtered)

    # Perform contrast stretching
    low_percentile = 22
    high_percentile = 97
    min_val, max_val = np.percentile(image_array, [low_percentile, high_percentile])
    stretched_image_array = (image_array - min_val) * 255 / (max_val - min_val)

    stretched_image_array = np.clip(stretched_image_array, 0, 255)
    stretched_image = Image.fromarray(stretched_image_array.astype(np.uint8))
    return stretched_image

# Image processing pipeline
def process_image(load_folder_path):
  for root, dirs, files in os.walk(load_folder_path):
    for file in files:
        image_path = os.path.join(root, file)
        image = Image.open(image_path)

        image_gray = gray_scaling(image)
        image_cropped = cropping(image_gray)
        image_rescaled = rescaling(image_cropped)
        image_normalized = normalize_intensity(image_rescaled)
        image_filtered = filter_gaussian(image_normalized)
        stretched_image = contrast_stretch(image_filtered)

        # Save processed image to google drive
        processed_folder_path = '/content/drive/MyDrive/Dataset_Processed'
        relative_path = os.path.relpath(root, load_folder_path)
        processed_folder = os.path.join(processed_folder_path, relative_path)
        os.makedirs(processed_folder, exist_ok=True)
        processed_image_path = os.path.join(processed_folder, file)
        stretched_image.save(processed_image_path)

# Access dataset from google drive
load_folder_path = '/content/drive/MyDrive/Dataset'

# Call image processing pipeline
process_image(load_folder_path)


Mounted at /content/drive
